In [61]:
import os
import glob
import ipywidgets as widgets
import zipfile as zf
import gzip
import shutil
from radiomics import featureextractor

In [82]:
dicom_dir_input = widgets.Text(
    value='',
    placeholder='Name of your compressed DICOM folder, end with .zip',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width='500px')
)

In [83]:
# widget to let the user enter the name of the dicom folder
display(dicom_dir_input)

Text(value='', description='String:', layout=Layout(width='500px'), placeholder='Name of your compressed DICOM…

In [67]:
# decompress dicom zip folder
dicom_file_name = dicom_dir_input.value.split('.')[0]
files = zf.ZipFile(dicom_dir_input.value, 'r')
files.extractall(dicom_file_name)
files.close()

In [68]:
# read dicom file and generate .nii.gz file using SimpleITK image reader
reader = sitk.ImageSeriesReader()
tmp_name = reader.GetGDCMSeriesFileNames(os.path.join(dicom_file_name, str(dicom_file_name)))
reader.SetFileNames(tmp_name)
image = reader.Execute()
sitk.WriteImage(image, dicom_file_name+'.nii.gz')

In [69]:
# decompress nii.gz file to nii file
with gzip.open(dicom_file_name+'.nii.gz', 'rb') as f_in:
    with open(dicom_file_name+'.nii', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [70]:
# feature extraction
settings = {}
settings['binWidth'] = 70
settings['resampledPixelSpacing'] = [5,5,5]
settings['interpolator'] = sitk.sitkNearestNeighbor
settings['normalize'] = True
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableImageTypes(Original={}, LoG={"sigma" : [4.0]}, Wavelet={})


# the parameters that are used in the final model
modeling_params = ['RV','MIN','SGE','BSY','SPAP','PA','MDR','SPH','FLA','SA','RMS']
# the function to compute final result using modeling_params
def joint_lr_model(dic):
    #P(PAH)=1/(EXP(-0.604*RV+0.11*MIN-0.16*SGE-0.226*BSY-1.897*SPAP-0.927*PA+0.748*MDR-0.614*SPH+0.306*FLA-0.036*SA-0.286*RMS+18.27)+1)
    odd = -0.604*dic['RV']+0.11*dic['MIN']-0.16*dic['SGE']-0.226*dic['BSY']-1.897*dic['SPAP']-0.927*dic['PA']+0.748*dic['MDR']-0.614*dic['SPH']+0.306*dic['FLA']-0.036*dic['SA']-0.286*dic['RMS']+18.27
    result = 1/exp(odd)+1
    return result

In [71]:
dicom_file_name = '45'

In [74]:
modeling_params_dic = {}
# extract features from pa.nrrd file
image_file = dicom_file_name+'.nii.gz'
mask_file = dicom_file_name+'_pa101.nrrd'
featureVector = extractor.execute(image_file, mask_file)
for param_name in modeling_params:
    if param_name in featureVector:
        modeling_params_dic[param_name] = featureVector[param_name]

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [76]:
featureVector

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '1.22.3'),
             ('diagnostics_Versions_SimpleITK', '2.1.1.2'),
             ('diagnostics_Versions_PyWavelet', '1.1.1'),
             ('diagnostics_Versions_Python', '3.8.5'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': [5, 5, 5],
               'interpolator': 1,
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True,
               'binWidth': 70}),
             ('diagnostics_Configuration_EnabledImageTypes',
              {

In [77]:
# extract features from mask-label.nrrd # 肺实质
mask_file = dicom_file_name+'_mask-label.nrrd'
featureVector2 = extractor.execute(image_file, mask_file)
for param_name in modeling_params:
    if param_name in featureVector2:
        modeling_params_dic[param_name] = featureVector[param_name]

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [79]:
featureVector2

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '1.22.3'),
             ('diagnostics_Versions_SimpleITK', '2.1.1.2'),
             ('diagnostics_Versions_PyWavelet', '1.1.1'),
             ('diagnostics_Versions_Python', '3.8.5'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': [5, 5, 5],
               'interpolator': 1,
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True,
               'binWidth': 70}),
             ('diagnostics_Configuration_EnabledImageTypes',
              {

In [ ]:
# collect the mean and std of needed features 
parameter_dics = {
    'original_firstorder_Minimum':
}

In [ ]:
# RV mean:41.8 std:9.527281401800353
# original_firstorder_Minimum: mean:-0.3196264678863637, std: 0.33672656523502387
# gldm_SmallDependenceLowGrayLevelEmphasis: mean:0.08525043123181816 std:0.01827983682244386
# ngtdm_Busyness: mean:1683.6258909918547 std:5023.316178714534
# SPAP: mean:55.69090909090909 std:27.990175372001012
# PA: mean:3.2348636363636363 std:0.6519024463727505
# original_shape_Maximum2DDiameterRow mean:208.04910237636366 std:27.00383979187061
# original_shape_Sphericity mean:0.4983680056863636 std:0.023067973310864284
# original_shape_Flatness mean:0.5319147852954547  std:0.05120626560217771
# original_shape_SurfaceArea mean:222388.73242990908 std:45253.3147554674
# original_firstorder_RootMeanSquared mean:0.5434065142772727 std:0.08810365068921959

In [ ]:
# user input parameter value
# RV mean:41.8 std:9.527281401800353 unit:mm
# spap: mean:55.69090909090909 std:27.990175372001012 unit: mmHg
# PA: mean:3.2348636363636363 std:0.6519024463727505 unit:cm

In [ ]:
# features from 肺实质, mask-label.nrrd
# original_shape_SurfaceArea mean:222388.73242990908 std:45253.3147554674 abbr: SA
# original_shape_Flatness mean:0.5319147852954547  std:0.05120626560217771 abbr: FLA
# original_shape_Sphericity mean:0.4983680056863636 std:0.023067973310864284 abbr: SPH

In [ ]:
# features from 肺血管，pa101.nrrd
# original_firstorder_Minimum: mean:-0.3196264678863637, std: 0.33672656523502387 abbr:MIN
# original_gldm_SmallDependenceLowGrayLevelEmphasis: mean:0.08525043123181816 std:0.01827983682244386 abbr:SGE
# original_ngtdm_Busyness: mean:1683.6258909918547 std:5023.316178714534 abbr:BSY
# original_shape_Maximum2DDiameterRow mean:208.04910237636366 std:27.00383979187061 abbr:MDR
# original_firstorder_RootMeanSquared mean:0.5434065142772727 std:0.08810365068921959 abbr:RMS

In [80]:
# modeling parameter dic
modeling_features_dic = {
    'RV':{'mean':41.8, 'std':9.527281401800353, 'unit':'mm', type:'user_input'},
    'SPAP':{'mean':55.69090909090909, 'std':27.990175372001012, 'unit':'mmHg', type:'user_input'},
    'PA':{'mean':3.2348636363636363, 'std':0.6519024463727505, 'unit':'cm', type:'user_input'},
    'SA':{'name':'original_shape_SurfaceArea', 'mean':222388.73242990908, 'std':45253.3147554674, type:'mask_label'},
    'FLA':{'name':'original_shape_Flatness', 'mean':0.5319147852954547, 'std':0.05120626560217771, type:'mask_label'},
    'SPH':{'name':'original_shape_Sphericity', 'mean':0.4983680056863636, 'std':0.023067973310864284, type:'mask_label'},
    'MIN':{'name':'original_firstorder_Minimum', 'mean':-0.3196264678863637, 'std':0.33672656523502387, type:'pa'},
    'SGE':{'name':'original_gldm_SmallDependenceLowGrayLevelEmphasis', 'mean':0.08525043123181816, 'std':0.01827983682244386, type:'pa'},
    'BSY':{'name':'original_ngtdm_Busyness','mean':1683.6258909918547,'std':5023.316178714534,type:'pa'},
    'MDR':{'name':'original_shape_Maximum2DDiameterRow','mean':208.04910237636366,'std':27.00383979187061,type:'pa'},
    'RMS':{'name':'original_firstorder_RootMeanSquared','mean':0.5434065142772727,'std':0.08810365068921959,type:'pa'}
}

In [92]:
# user input parameters
rv_input = widgets.FloatText(
    description='RV:',
    disabled=False
)
rv_unit = widgets.Label('mm')
rv_box = widgets.HBox([rv_input, rv_unit])


spap_input = widgets.FloatText(
    description='SPAP:',
    disabled=False
)
spap_unit = widgets.Label('mmHg')
spap_box = widgets.HBox([spap_input, spap_unit])


pa_input = widgets.FloatText(
    description='PA:',
    disabled=False
)
pa_unit = widgets.Label('cm')
pa_box = widgets.HBox([pa_input, pa_unit])

boxes = [rv_box, spap_box, pa_box]

In [106]:
for box in boxes:
    display(box)